<a href="https://colab.research.google.com/github/Adjinti/KalAIEDdoscope/blob/main/nico/24_06_11_L3_customer_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is to follow the course about multiagents and crewAI on
https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/6/multi-agent-customer-support-automation-(code)

Then get my badge here. :)
https://learn.crewai.com/

# L3: Multi-agent Customer Support Automation
In this lesson, you will learn about the six key elements which help make Agents perform even better:

- Role Playing  
- Focus  
- Tools  
- Cooperation  
- Guardrails
- Memory

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
from crewai import Agent, Task, Crew

Attention: collab use a diferent way to store private environment variable than Jupyter.

In [5]:
import os
# from utils import get_openai_api_key
# openai_api_key = get_openai_api_key()

from google.colab import userdata
openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Role Playing, Focus and Cooperation

In [6]:
support_agent = Agent(
  role="Senior Support Representative",
  goal="Be the most friendly and helpful "
      "support representative in your team",
  backstory=(
      "You work at crewAI (https://crewai.com) and "
      " are now working on providing "
      "support to {customer}, a super important customer "
      " for your company."
      "You need to make sure that you provide the best support!"
      "Make sure to provide full complete answers, "
      " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [7]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
      "best support quality assurance in your team",
	backstory=(
      "You work at crewAI (https://crewai.com) and "
      "are now working with your team "
      "on a request from {customer} ensuring that "
      "the support representative is "
      "providing the best support possible.\n"
      "You need to make sure that the support representative "
      "is providing full"
      "complete answers, and make no assumptions."
	),
	verbose=True
)

## Tools, Guardrails and Memory
### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

**Different Ways to Give Agents Tools**

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

In [10]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [11]:
# search_tool = SerperDevTool()
# scrape_tool = ScrapeWebsiteTool()

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

### Creating Tasks
- You are passing the Tool on the Task Level.

**Note**: Task Tools override the Agent Tools.

In [12]:
inquiry_resolution = Task(
    description=(
      "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
      "{person} from {customer} is the one that reached out. "
	  	"Make sure to use everything you know "
      "to provide the best support possible."
	  	"You must strive to provide a complete "
      "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
      "customer's inquiry that addresses "
      "all aspects of their question.\n"
      "The response should include references "
      "to everything you used to find the answer, "
      "including external data or solutions. "
      "Ensure the answer is complete, "
      "leaving no questions unanswered, and maintain a helpful and friendly "
      "tone throughout."
    ),
	tools=[docs_scrape_tool],
  agent=support_agent,
)

In [13]:
quality_assurance_review = Task(
    description=(
      "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
      "Ensure that the answer is comprehensive, accurate, and adheres to the "
		  "high-quality standards expected for customer support.\n"
      "Verify that all parts of the customer's inquiry "
      "have been addressed "
	  	"thoroughly, with a helpful and friendly tone.\n"
      "Check for references and sources used to "
      " find the information, "
      "ensuring the response is well-supported and "
      "leaves no questions unanswered."
    ),
    expected_output=(
      "A final, detailed, and informative response "
      "ready to be sent to the customer.\n"
      "This response should fully address the "
      "customer's inquiry, incorporating all "
      "relevant feedback and improvements.\n"
		  "Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [14]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [15]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide the best support possible to DeepLearningAI by thoroughly understanding their question and providing a detailed response. I should use the tool "Read website content" to gather information on setting up a Crew and adding memory.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

Assembling and Activating Your CrewAI Team - crewAI
 Skip to content
 crewAI
 Assembling and Ac

In [16]:
from IPython.display import Markdown
Markdown(result)

Hello Andrew Ng from DeepLearningAI,

Thank you for reaching out for assistance with setting up a Crew and adding memory to it. I have gathered information from our documentation on https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/ to provide you with detailed guidance on this matter.

To add memory to your Crew in CrewAI, you will need to follow these steps:

Step 1: Assemble Your Agents
- Define your agents with distinct roles, backstories, and enhanced capabilities like memory usage.
- You can create agents with memory by specifying the 'memory=True' parameter when defining the agent. This will allow the agent to retain information and enhance its performance.

Step 2: Define the Tasks
- Detail the specific objectives for your agents, including tasks that require memory usage.
- Ensure that the tasks assigned to agents with memory capabilities leverage this feature effectively to enhance their performance.

Step 3: Form the Crew
- Combine your agents into a crew and configure memory settings for the crew.
- When forming the crew, set the 'memory=True' parameter to enable memory capabilities for the entire crew.

Step 4: Kick It Off
- Initiate the process with your enhanced crew ready to collaborate and leverage their memory capabilities for successful project outcomes.
- Input variables will be interpolated into the agents and tasks for a personalized approach.

By following these steps, you can successfully add memory to your Crew in CrewAI and optimize its performance. If you have any further questions or need additional assistance, please feel free to reach out to us.

We are here to help you make the most of your CrewAI experience and achieve your goals efficiently.

Best regards,
[Your Name]
Senior Support Representative
crewAI